In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

2023-04-26 14:29:15.889924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 14:29:15.986997: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-26 14:29:16.378409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-26 14:29:16.378470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-26 14:29:16.924298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 14:29:16.924498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 14:29:16.924638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 14:29:17.260625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 14:29:17.260785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Simultaneously Estimate Translation, Rotation, and Distortion

## $\mathbf{A} = [\hat{X}_{ij}, \hat{m}_{ij}] = 
\begin{bmatrix}
% x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
\end{bmatrix}$

We can use newton-raphson to find A

<!-- ## $y_i = \mathbf{h}(y_j, \hat{X}_{ij}, \hat{m}_{ij}) + \mathbf{H}_m \delta m + \mathbf{H}_x \delta x + \text{H.O.T.}$ -->

## $y_i = \mathbf{h}(y_j, \hat{A}_{ij}) + \mathbf{H}_A \delta A + \text{H.O.T.}$

## $\mathbf{H}_A \in \mathbb{R}^{4N \times 12} $

### $\mathbf{H}_A = [H_X, H_m]$

# Run Demo

In [51]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# # movement in x
# old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
# m_hat = np.array([3, 0, 0., 0., 0., 0])
# # m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
# gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
# m_hat = np.array([3, -1, 0., 0., 0., -1])
m_hat = np.array([3, -1, 0., 0., 0., -0.86]) #FOR DEBUG-- deform a little extra
gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)

#downsample
old_cloud = old_cloud[::5,:]

In [54]:
from remove_motion_basic import linear_correction_old as lc 

#apply ground truth distortion according to m_hat
new_cloud = lc(old_cloud, m_hat) 
#set ground truth transform between clouds
X_gt = np.array([0.5, 0.5, 0.03, 0.004, 0.05, 0.06])

# trans = X_gt[:3]
# rot = R_tf(X_gt[3:]).numpy()

# new_cloud = (new_cloud @ rot) + trans

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "raw point clouds")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

### Attempt to solve with basic Newton-Raphson

In [55]:
from linear_corrector import LC
pc1 = old_cloud
pc2  = new_cloud
m_hat0 = np.array([0, 0, 0, 0, 0, 0.])
dc = LC(cloud1 = pc2, cloud2 = pc1, fid = 50, niter = 100, draw = True, m_hat0 = m_hat0,  mnp = 25, RM = False)
ViewInteractiveWidget(dc.plt.window)


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.0159759521484375 sec  to apply motion profile
took 0.0019893646240234375 sec to get H
m_hat:  [ 4.26354930e-02 -2.59516209e-01 -1.16110547e-04  5.41618647e-07
  2.80802124e-06 -7.31899664e-02]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.014623403549194336 sec  to apply motion profile
took 0.002057790756225586 sec to get H
m_hat:  [ 7.89496005e-02 -4.47333872e-01 -2.18919740e-04  1.27463172e-05
  3.02537828e-05 -1.46405607e-01]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
took 0.014596700668334961 sec  to apply motion profile
took 0.002062559127807617 sec to get H
m_hat:  [ 1.12228602e-01 -6.01998091e-01 -1.45812621e-04 -2.84962262e-05
  3.52939096e-05 -2.22076565e-01]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.019916772842407227 sec  to apply motion profile
took 0.0021207332611083984 sec to get H
m_hat:  [ 1.29235893e-01 -7.53395915e-01 -2.28239922e-04 -5.49516335e-05
  5.85396920e-05 -3.07879239e-01]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
took 0.01633381843566

took 0.005560636520385742 sec to get H
m_hat:  [ 2.99232864e+00 -1.02106822e+00 -8.26263567e-05 -2.04631870e-05
  3.03024972e-05 -8.58638406e-01]
~~~~~~~~~~~Iteration  36 ~~~~~~~~~~
took 0.01620197296142578 sec  to apply motion profile
took 0.0023424625396728516 sec to get H
m_hat:  [ 2.99458098e+00 -1.01583087e+00 -6.61782833e-05 -1.72261534e-05
  2.11466449e-05 -8.59068751e-01]
~~~~~~~~~~~Iteration  37 ~~~~~~~~~~
took 0.014799118041992188 sec  to apply motion profile
took 0.0023152828216552734 sec to get H
m_hat:  [ 2.99624968e+00 -1.01178789e+00 -5.55672159e-05 -1.49135230e-05
  1.48410854e-05 -8.59382331e-01]
~~~~~~~~~~~Iteration  38 ~~~~~~~~~~
took 0.014405965805053711 sec  to apply motion profile
took 0.0023784637451171875 sec to get H
m_hat:  [ 2.99746680e+00 -1.00869536e+00 -4.81193747e-05 -1.17847849e-05
  1.06798279e-05 -8.59607100e-01]
~~~~~~~~~~~Iteration  39 ~~~~~~~~~~
took 0.01609659194946289 sec  to apply motion profile
took 0.0023605823516845703 sec to get H
m_hat:  [ 2

m_hat:  [ 3.00000048e+00 -9.99997616e-01  8.32777289e-07 -5.34343087e-07
 -7.39877350e-07 -8.60000014e-01]
~~~~~~~~~~~Iteration  72 ~~~~~~~~~~
took 0.015900611877441406 sec  to apply motion profile
took 0.0024406909942626953 sec to get H
m_hat:  [ 3.00000048e+00 -9.99997735e-01  1.16407386e-06 -6.85082910e-07
 -1.13937040e-06 -8.59999955e-01]
~~~~~~~~~~~Iteration  73 ~~~~~~~~~~
took 0.01501154899597168 sec  to apply motion profile
took 0.002390146255493164 sec to get H
m_hat:  [ 3.00000000e+00 -9.99997914e-01  1.40205088e-06 -7.29211138e-07
 -1.37804432e-06 -8.59999955e-01]
~~~~~~~~~~~Iteration  74 ~~~~~~~~~~
took 0.01668262481689453 sec  to apply motion profile
took 0.0025331974029541016 sec to get H
m_hat:  [ 3.00000024e+00 -9.99997854e-01  1.60361037e-06 -7.26003066e-07
 -1.53751000e-06 -8.59999895e-01]
~~~~~~~~~~~Iteration  75 ~~~~~~~~~~
took 0.01685500144958496 sec  to apply motion profile
took 0.0025763511657714844 sec to get H
m_hat:  [ 3.00000000e+00 -9.99997556e-01  1.76342428

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [48]:
# ##DEBUG: make sure that correspondeces between old and new cloud are good
# pc1_theta = dc.c2s(pc1)[:,1]
# pc2_theta = dc.c2s(pc2)[:,1]
# from matplotlib import pyplot as plt
# fig, ax = plt.subplots()
# ax.plot(pc1_theta)
# ax.plot(pc2_theta)

In [ ]:
A_hat = np.array([0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 1, r=3.5))

runlen = 5
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("A_hat = ", A_hat)
    
    #decompose A_hat into X_hat and m_hat
    X_hat = A_hat[:6]
    m_hat = A_hat[6:]

    #apply last estimate of distortion correction
#     print("m_hat", m_hat)
    y_j_undistort = lc(y_j, m_hat)

    #apply last rigid transform
    rot = R_tf(-X_hat[3:]).numpy()
    trans = X_hat[:3]
    y_j_undistort = (y_j_undistort @ rot) + trans
    
#     print(rot, trans)  
#     print("\n y_i \n",np.shape(y_i), "\n", y_i[:3])
#     print("y_j_undistort \n",np.shape(y_j_undistort), "\n", y_j_undistort[:3])

    
    #get H = [H_X, H_m]
    #get jacobain of distortion correction function
    H_m = dc.get_H_m(y_j_undistort, m_hat) 
#     print("H_m:", np.shape(H_m), "\n", H_m[:10])
    
    #get jacobian of rigid transform function 
    H_x = jacobian_tf(tf.transpose(tf.convert_to_tensor(y_j_undistort, tf.float32)), tf.convert_to_tensor(X_hat[3:], tf.float32)) # shape = [num of corr * 3, 6]
    #need to append on a row of zeros since we are working with homogeneous coordinates!
    H_x = tf.reshape(H_x, (tf.shape(H_x)[0]//3, 3, 6)) # -> need shape [#corr//4, 4, 6]
#     print("before:", np.shape(H_x), "\n", H_x[0])
    H_x = tf.concat([H_x, tf.zeros([len(H_x),1,6])], axis = 1)
    H_x = tf.reshape(H_x, (-1, 6))
#     print("after:", np.shape(H_x), "\n", H_x[:10])
    H_x = H_x.numpy()
        
    #delta_A =  ((H^T*H)^-1)(H^T)(yi - yj_undistort)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
#     print("residual", np.shape(residual), "\n", residual)
    
    H = np.append(H_x, H_m, axis = 1)
#     print("H: \n", np.shape(H))
    
    delta_A = np.linalg.pinv(H.T @ H) @ H.T @ residual
    print("delta_A", delta_A)
    #augment rigid transform components
    A_hat[:6] +=   delta_A[:6]
    #augment distortion components
    A_hat[6:9] -= delta_A[6:9]
    A_hat[9:] += delta_A[9:]
    
    #plot updated cloud2
    color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = color, r=3.5))
#     disp.append(Points(y_j_undistort[:,:3], c = "#a65852 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "12 State Solution")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
A_hat =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
delta_A [ 0.06159182  0.27704321 -0.00145194  0.00244503  0.00453987  0.1725553
  0.06159182  0.27704321 -0.00145194 -0.00244503 -0.00453987 -0.1725553 ]
~~~~ iteration  1 ~~~~~~~~~~~
A_hat =  [ 0.06159182  0.27704321 -0.00145194  0.00244503  0.00453987  0.1725553
 -0.06159182 -0.27704321  0.00145194 -0.00244503 -0.00453987 -0.1725553 ]
delta_A [ 1.62454826e-02  4.03530005e-01 -5.35154748e-04 -1.94378830e+00
 -3.82528115e-01  1.31772603e-01  1.62454826e-02  4.03530005e-01
 -5.35154748e-04 -1.94210161e+00 -3.79684906e-01  3.62724367e-02]
~~~~ iteration  2 ~~~~~~~~~~~
A_hat =  [ 0.0778373   0.68057322 -0.00198709 -1.94134328 -0.37798825  0.3043279
 -0.0778373  -0.68057322  0.00198709 -1.94454664 -0.38422477 -0.13628286]
delta_A [-0.23421648  0.96446303  0.63607353 -0.21743208 -0.15925499  0.08500089
 -0.23421648  0.96446303  0.63607353 -0.47829786 -0.26615443 -0.29718653]
~~~~ iteration  3 ~~~~~~~~~~~
A_hat =  [